<a href="https://colab.research.google.com/github/matpg/RL-Agent-for-Unreal-Engine/blob/main/Codigo_Procgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Descarga de Librerias

In [1]:
!pip install procgen
!git clone https://github.com/openai/train-procgen.git
!conda env update --name train-procgen --file train-procgen/environment.yml
!conda activate train-procgen
!pip install https://github.com/openai/baselines/archive/9ee399f5b20cd70ac0a871927a6cf043b478193f.zip
!pip install -e train-procgen
!pip install mpi4py

     |████████████████████████████████| 39.9MB 76kB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 3.3MB 51.4MB/s 
     |████████████████████████████████| 204kB 56.7MB/s 
     |████████████████████████████████| 22.2MB 1.3MB/s 
     |████████████████████████████████| 665kB 49.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
Cloning into 'train-procgen'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 387 (delta 2), reused 12 (delta 2), pack-reused 368
Receiving objects: 100% (387/387), 71.37 MiB | 24.53 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Checking out files: 100% (306/306), done.
/bin/bash: conda: command not found
/bin/bash: c

### Comprobación de carga de librerias

In [2]:
from procgen import ProcgenGym3Env

### Entrenamiento en entorno Maze de Procgen

In [6]:
%tensorflow_version 1.x
!python -m train_procgen.train --env_name maze

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'train_procgen.train' found in sys.modules after import of package 'train_procgen', but prior to execution of 'train_procgen.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Logging to /tmp/procgen
creating environment
creating tf session


2020-11-03 22:55:59.121261: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-11-03 22:55:59.121502: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3070680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-03 22:55:59.121535: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-11-03 22:55:59.123530: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-03 22:55:59.235946: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA nod

In [10]:
!python -m train_procgen.train --help

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'train_procgen.train' found in sys.modules after import of package 'train_procgen', but prior to execution of 'train_procgen.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: train.py [-h] [--env_name ENV_NAME] [--num_envs NUM_ENVS]
                [--distribution_mode {easy,hard,exploration,memory,extreme}]
                [--num_levels NUM_LEVELS] [--start_level START_LEVEL]
                [--test_worker_interval TEST_WORKER_INTERVAL]
                [--timesteps_per_proc TIMESTEPS_PER_PROC]

Process procgen training arguments.

optional arguments:
  -h, --help            show this help message and exit
  --env_name ENV_NAME
  --num_envs NUM_ENVS
  --distribution_mode {easy,hard,exploration,memory,extreme}
  --num_levels NUM_LEVELS
  --start_level START_LEVEL
  --test_worker_interval TEST_WORKER_INTERVAL
  --timesteps_per_proc TIMESTEPS_PER_PROC
[3504aeb30f06:00669] *** Process received signal ***
[

In [ ]:
# Eenvironement for learning
# Use a separate environement for evaluation
env = ProcgenGym3Env(num=8, env_name="maze")
print(env)
train_env = env


<CEnv lib_path=/usr/local/lib/python3.6/dist-packages/procgen/data/prebuilt/libenv.so options={'center_agent': True, 'use_generated_assets': False, 'use_monochrome_assets': False, 'restrict_themes': False, 'use_backgrounds': True, 'paint_vel_info': False, 'distribution_mode': 1, 'env_name': 'maze', 'num_levels': 0, 'start_level': 0, 'num_actions': 15, 'use_sequential_levels': False, 'debug_mode': 0, 'rand_seed': 1406393713, 'num_threads': 4, 'render_human': False, 'resource_root': '/usr/local/lib/python3.6/dist-packages/procgen/data/assets/'}>


In [ ]:
def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env = RGBImgObsWrapper(env) # Get pixel observations
        env = ImgObsWrapper(env)
        env = Monitor(env, './monitor_ppo2')
        # Important: use a different seed for each environment
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init

In [ ]:
n_process = 8
env = DummyVecEnv([make_env(env_id, i) for i in range(n_process)])
train_env = VecNormalize(env)

In [ ]:
%cd /content/gym-minigrid/

/content/gym-minigrid


In [ ]:
# Use a separate environement for evaluation
# eval_env = gym.make('CartPole-v1')

# Random Agent, before training
# mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

# print(f"mean_reward for random agent:{mean_reward:.2f} +/- {std_reward:.2f}")

### Definicion de Modelo del Agente

In [ ]:
agent_model = PPO2(
    'CnnPolicy',
     train_env,
      ent_coef=0.0,
       nminibatches=32,
        noptepochs=10,
         verbose=1,
         )

### Evaluar Agente Ingenuo/Aleatorio

In [ ]:
def evaluate(model, eval_env, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    # env = gnwrapper.LoopAnimation(model.get_env()) # Start Xvfb
    env = eval_env
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            # env.render()
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
        

    # env.display()
    max_episode_reward = np.amax(all_episode_rewards)
    mean_episode_reward = np.mean(all_episode_rewards)
    min_episode_reward = np.amin(all_episode_rewards)
    print(
        "Max reward:", max_episode_reward,
        "Mean reward:", mean_episode_reward,
        "Min reward:", min_episode_reward,
         "Num episodes:", num_episodes)

    return max_episode_reward, mean_episode_reward, min_episode_reward

In [ ]:
# Random Agent, before training using property
# from stable_baselines.common.evaluation import evaluate_policy
max_r, mean_r, min_r = evaluate(agent_model, eval_env, num_episodes=100)

# print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Max reward: 0 Mean reward: 0.0 Min reward: 0 Num episodes: 100


### Entrenar Agente

In [ ]:
agent_model.learn(total_timesteps=10000, log_interval=10)
train_env.close()

--------------------------------------
| approxkl           | 0.011721635   |
| clipfrac           | 0.175         |
| ep_len_mean        | 58.6          |
| ep_reward_mean     | 0             |
| explained_variance | 0.636         |
| fps                | 258           |
| n_updates          | 1             |
| policy_entropy     | 1.9330553     |
| policy_loss        | -0.022883493  |
| serial_timesteps   | 128           |
| time_elapsed       | 1.98e-05      |
| total_timesteps    | 1024          |
| value_loss         | 0.00035427717 |
--------------------------------------


### Evaluar Agente Entrenado

In [ ]:
# Trained Agent, after training using property
# from stable_baselines.common.evaluation import evaluate_policy
max_r, mean_r, min_r = evaluate(agent_model, eval_env, num_episodes=100)

Max reward: 0.4666666666666667 Mean reward: 0.004666666666666667 Min reward: 0.0 Num episodes: 100


In [ ]:
import os

# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

### Extras

In [ ]:
import gym_minigrid

env = gym.make('MiniGrid-SimpleCrossingEnvUmaze-v0')
for i_episode in range(3):
    obs = env.reset()
    for t_steps in range(100):
        # env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        print(obs)
        if done:
            print("Episode finished after {} timesteps".format(t_steps+1))
            break
env.close()

In [ ]:
!apt update && apt install xvfb
!pip install gym-notebook-wrapper

In [ ]:
# Random Agent, before training using function
mean_reward_before_train = evaluate(agent_model, eval_env, num_episodes=100)

In [ ]:
!pip install tensorboard==2.0.0

In [ ]:
# Trained Agent

mean_reward, std_reward = evaluate_policy(agent_model, train_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
!tensorboard --logdir ./ppo2_minigriddoorkey/ --host localhost 